In [25]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score, classification_report
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd


## Carga de datos

In [27]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [28]:
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}
y_train = newsgroups_train.target

In [29]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

____________________________________________________________________________________________________

In [30]:
# Elegimos 5 documentos al azar
SEED = 1234
rng = np.random.default_rng(SEED)
anchor_ids = rng.choice(X_train.shape[0], size=5, replace=False)
anchor_ids

array([11047, 11175,  4301, 11076,  1938])

In [31]:
topk = 5

## 1. ¿Tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación? ##

#### Sí, sí tiene sentido, pero algunos temas son más similares entre ellos, otros son más diferentes e inconfundibles. Lo otro es si uno justo se fija en un documento que esté relativamente lejos del "centro de masa", 4301 tiene una lista de fechas de sinagoga y se confunde con listas de fechas de deportes.

In [32]:
for a in anchor_ids:
    sims = cosine_similarity(X_train[a], X_train).ravel()
    order = np.argsort(sims)[::-1]
    top_idx = [i for i in order if i != a][:topk]

    print(f'Ancla {a}:', end=" ")
    print(newsgroups_train.target_names[y_train[a]])

    for i in top_idx:
        print(f'Vecino {i}:', end=" ")
        print(newsgroups_train.target_names[y_train[i]])
    print('____________')


Ancla 11047: rec.motorcycles
Vecino 7642: rec.motorcycles
Vecino 5804: rec.motorcycles
Vecino 2877: rec.motorcycles
Vecino 2944: rec.motorcycles
Vecino 8134: rec.motorcycles
____________
Ancla 11175: comp.sys.mac.hardware
Vecino 6624: comp.sys.mac.hardware
Vecino 6057: comp.graphics
Vecino 8487: sci.electronics
Vecino 10591: comp.os.ms-windows.misc
Vecino 9448: comp.os.ms-windows.misc
____________
Ancla 4301: talk.politics.mideast
Vecino 3520: rec.sport.hockey
Vecino 4352: rec.sport.baseball
Vecino 8829: rec.sport.hockey
Vecino 3970: rec.sport.hockey
Vecino 372: misc.forsale
____________
Ancla 11076: misc.forsale
Vecino 327: misc.forsale
Vecino 9490: misc.forsale
Vecino 1508: misc.forsale
Vecino 4648: misc.forsale
Vecino 2083: misc.forsale
____________
Ancla 1938: rec.motorcycles
Vecino 8467: rec.motorcycles
Vecino 1849: rec.motorcycles
Vecino 11019: rec.motorcycles
Vecino 3215: rec.motorcycles
Vecino 6889: rec.motorcycles
____________


In [33]:

for a in anchor_ids:
    sims = cosine_similarity(X_train[a], X_train).ravel()
    order = np.argsort(sims)[::-1]
    top_idx = [i for i in order if i != a][:topk]

    print(f'Ancla {a}:')
    print('____________')
    print(newsgroups_train.data[a])
    print('_____________________________________________________________________________________________________')
    print('_____________________________________________________________________________________________________')

    for i in top_idx:
        print(f'Vecino {i}:')
        print('_____________')
        print(newsgroups_train.data[i])
        print('_____________________________________________________________________________________________________')
        print('_____________________________________________________________________________________________________')
    print('_____________________________________________________________________________________________________')
    print('_____________________________________________________________________________________________________')



Ancla 11047:
____________



 I hate to  admit this but there does seem to be some sort of twisted logic
to this approach. It's the bikers against the world and the dogs are just 
another worthless adversary. So remember to wear at least calf height leather
boots, ( in case the dog gets lucky and sinks his teeth into your
attacking foot) and go for the gusto, If that dog doesn't retreat from the
street with his tail between his legs next time you see it then you really
haven't done your bit for all your fellow bikers.


Sorry I can't go this far, A dog against and armored cage just doesn't
seem like a fair fight.


after all it is a dog eat dog world 
_____________________________________________________________________________________________________
_____________________________________________________________________________________________________
Vecino 7642:
_____________
I'm a biker and a dog-lover.

First and foremost, I want to mention some common sense.  If it's a choice
betw

## 2. 1-NN


In [34]:
# Usamos NearestNeighbors con métrica coseno (brute force) para 1-NN
nn = NearestNeighbors(n_neighbors=1, metric='cosine', algorithm='brute')
nn.fit(X_train)

# Para cada doc de test, obtenemos su vecino más cercano
dist, idx = nn.kneighbors(X_test, return_distance=True)
# Nota: similitud coseno = 1 - distancia_coseno
y_pred_proto = y_train[idx.ravel()]

f1_proto = f1_score(y_test, y_pred_proto, average='macro')
print("F1-macro (prototipos 1-NN por coseno):", f1_proto)

print("\nReporte de clasificación (resumen):")
print(classification_report(y_test, y_pred_proto, target_names=newsgroups_test.target_names, digits=3))

F1-macro (prototipos 1-NN por coseno): 0.504078533146506

Reporte de clasificación (resumen):
                          precision    recall  f1-score   support

             alt.atheism      0.366     0.508     0.425       319
           comp.graphics      0.544     0.481     0.510       389
 comp.os.ms-windows.misc      0.506     0.457     0.480       394
comp.sys.ibm.pc.hardware      0.340     0.538     0.417       392
   comp.sys.mac.hardware      0.535     0.499     0.516       385
          comp.windows.x      0.701     0.592     0.642       395
            misc.forsale      0.629     0.462     0.533       390
               rec.autos      0.607     0.518     0.559       396
         rec.motorcycles      0.635     0.515     0.569       398
      rec.sport.baseball      0.645     0.537     0.586       397
        rec.sport.hockey      0.748     0.722     0.735       399
               sci.crypt      0.552     0.588     0.570       396
         sci.electronics      0.531     0.328  

## 3. Naïve Bayes

#### Se probaron varios valores de alfa (0,1 ; 0,5 ; 1 ; 2) Complement es un poco mejor que Multinomial y los dos son mejores que 1-NN.

In [35]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB(alpha=0.1)
clf.fit(X_train, y_train)

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

f1_score(y_test, y_pred, average='macro')


0.6564514103512165

In [36]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = ComplementNB(alpha=0.5)
clf.fit(X_train, y_train)

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

f1_score(y_test, y_pred, average='macro')

0.6961156947315815

## 4. Similaridad entre palabras ##

#### Si el término es frecuente en los temas del corpus, anda bien (electrónica religión,...). Si no, no.

In [37]:
vocab = tfidfvect.vocabulary_

In [38]:
# Transponemos TF–IDF de train
TD = X_train.T

def similar_terms(term, topk=5):
    term = term.lower()
    if term not in vocab:
        return []
    i = vocab[term]
    sims = cosine_similarity(TD[i], TD).ravel()
    order = np.argsort(sims)[::-1]
    # saltamos el propio término (posición 0)
    neigh = [(idx2word[j], float(sims[j])) for j in order[1:topk+1]]
    return neigh

manual_terms = ["satan", "dune", "asimov", "pcb", "cpu"]  
pairs = {t: similar_terms(t) for t in manual_terms}
pairs


{'satan': [('freewill', 0.42553896858010287),
  ('angels', 0.4012354653095535),
  ('tells', 0.25937187117279664),
  ('rightous', 0.2300620932982495),
  ('parr', 0.2300620932982495)],
 'dune': [('2061', 1.0),
  ('compleat', 1.0),
  ('dragonslayer', 1.0),
  ('sowrds', 1.0),
  ('extratresstials', 1.0)],
 'asimov': [('uproarious', 0.8407328446334369),
  ('humorist', 0.8407328446334369),
  ('vonnegut', 0.8116223286110549),
  ('humanists', 0.7323422152266449),
  ('funeral', 0.6739998180163189)],
 'pcb': [('futurenet', 0.6184327515890164),
  ('maarten', 0.5013449372390895),
  ('pads', 0.4965559048941749),
  ('soldered', 0.31247942687837305),
  ('import', 0.2733900821665188)],
 'cpu': [('dislodge', 0.37392870074924806),
  ('heatsink', 0.3211451037831309),
  ('cooler', 0.3008863591278762),
  ('bending', 0.2750774424719198),
  ('computationally', 0.26232018876733043)]}

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.
